In [1]:
import sys
sys.path.append("../src")
from plugin_write_and_run import *

In [2]:
%%write_and_run ../src/main.py
#!/usr/bin/python

import sys
sys.path.append("../src")
import gym
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import wandb
from config import *
from replay_buffer import *
from networks import *
from agent import *

In [3]:
%%write_and_run -a ../src/main.py

config = dict(
  learning_rate_actor = ACTOR_LR,
  learning_rate_critic = ACTOR_LR,
  batch_size = BATCH_SIZE,
  architecture = "DDPG",
  infra = "Ubuntu",
  env = ENV_NAME
)

In [4]:
%%write_and_run -a ../src/main.py

wandb.init(
  project=f"tensorflow2_ddpg_{ENV_NAME.lower()}",
  tags=["DDPG", "FCL", "RL"],
  config=config,
)

wandb: Currently logged in as: anton-ai (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
%%write_and_run -a ../src/main.py

env = gym.make(ENV_NAME)
agent = Agent(env)

scores = []
evaluation = False

In [6]:
%%write_and_run -a ../src/main.py

if PATH_LOAD is not None:
    print("loading weights")
    observation = env.reset()
    action = agent.actor(observation[None, :])
    agent.target_actor(observation[None, :])
    agent.critic(observation[None, :], action)
    agent.target_critic(observation[None, :], action)
    agent.load()
    print(agent.replay_buffer.buffer_counter)
    print(agent.replay_buffer.n_episodes)
    print(agent.noise)

In [7]:
%%write_and_run -a ../src/main.py

for _ in tqdm(range(MAX_GAMES)):
    start_time = time.time()
    states = env.reset()
    done = False
    score = 0
    while not done:
        action = agent.get_action(states, evaluation)
        new_states, reward, done, info = env.step(action)
        score += reward
        agent.add_to_replay_buffer(states, action, reward, new_states, done)
        agent.learn()
        states = new_states
        
    agent.replay_buffer.update_n_games()
    
    scores.append(score)
    
    wandb.log({'Game number': agent.replay_buffer.n_games, '# Episodes': agent.replay_buffer.buffer_counter, 
               "Average reward": round(np.mean(scores[-10:]), 2), \
                      "Time taken": round(time.time() - start_time, 2)})

    if (_ + 1) % EVALUATION_FREQUENCY == 0:
        states = env.reset()
        evaluation = True
        score = 0
        done = False
        while not done:
            action = agent.get_action(states, evaluation)
            new_states, reward, done, info = env.step(action)
            score += reward
            states = new_states
        wandb.log({'Game number': agent.replay_buffer.n_games, 
                   '# Episodes': agent.replay_buffer.buffer_counter, 
                   'Evaluation score': score})
        evaluation = False
     
    if (_ + 1) % SAVE_FREQUENCY == 0:
        print("saving...")
        agent.save()
        print("saved")

  7%|▋         | 18/250 [01:53<24:26,  6.32s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-94bfbd7604e9>", line 11, in <module>
    agent.learn()
  File "../src/agent.py", line 124, in learn
    critic_gradient = tape.gradient(critic_loss,
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py", line 1080, in gradient
    flat_grad = imperative_grad.imperative_grad(
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/imperative_grad.py", line 71, in imperative_grad
    return pywrap_tfe.TFE_Py_TapeGradient(
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py", line 162, in _gradient_function
    return grad_fn(mock_op, *out_grads)
  File 

TypeError: object of type 'NoneType' has no len()